## Seminar: Monte-carlo tree search

Monte Carlo tree search (MCTS) is a heuristic search algorithm, which shows cool results in challenging domains such as Go and chess. The algorithm builds a search tree, iteratively traverses it, and evaluates its nodes using a Monte-Carlo simulation.

In this notebook, we'll implement a MCTS planning and use it to solve some Gym envs.

![image.png](https://i.postimg.cc/6QmwnjPS/image.png)

__How it works?__
We just start with an empty tree and expand it. There are several common procedures.

__1) Selection__
Starting from the root, recursively select the node that corresponds to the tree policy.  

There are several options for tree policies, which we saw earlier as exploration strategies: epsilon-greedy, Thomson sampling, UCB-1. It was shown that in MCTS, UCB-1 achieves a good result. Further, we will consider the one, but you can try to use others.

Following the UCB-1 tree policy, we will choose an action that, on one hand, we expect to have the highest return, and on the other hand, we haven't explored much.

$$
\dot{a} = argmax_{a} \dot{Q}(s, a)
$$

$$
\dot{Q}(s, a) = Q(s, a) + C_p \sqrt{\frac{2 \log {N}}{n_a}}
$$

where:
- $N$ - number of times we have visited state $s$,
- $n_a$ - number of times we have taken action $a$,
- $C_p$ - exploration balance parameter, which is performed between exploration and exploitation.

Using Hoeffding inequality for rewards $R \in [0,1]$ it can be shown that optimal $C_p = 1/\sqrt{2}$. For rewards outside this range, the parameter should be tuned. You can experiment with other values.

__2) Expansion__
After the selection procedure, we can achieve a leaf node or node in which we don't complete actions. In this case, we expand the tree by feasible actions and get new state nodes.

__3) Simulation__
How we can estimate node Q-values? The idea is to estimate action values for a given _rollout policy_ by averaging the return of many simulated trajectories from the current node. Simply, we can play with random or some special policy or use some model that can estimate it.

__4) Backpropagation__
The reward of the last simulation is backed up through the traversed nodes and propagates Q-value estimations, upwards to the root.

$$
Q({\text{parent}}, a) = r + \gamma \cdot Q({\text{child}}, a)
$$

In [ ]:
try:
    import google.colab
    COLAB = True
except ModuleNotFoundError:
    COLAB = False
    pass

if COLAB:
    !pip -q install "gymnasium[classic-control, atari, accept-rom-license]"
    !pip -q install piglet
    !pip -q install imageio_ffmpeg
    !pip -q install moviepy==1.0.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 14.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.5/67.5 kB 2.2 MB/s eta 0:00:00


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
from itertools import count
%matplotlib inline

In [ ]:
env_name = 'CartPole-v1'
# env_name = 'MountainCar-v0'
# env_name = 'Acrobot-v1'

---

But before we do that, we first need to make a wrapper for Gym environments to allow saving and loading game states to facilitate backtracking.

In [ ]:
import gymnasium as gym
from gymnasium.core import Wrapper
from pickle import dumps, loads
from collections import namedtuple

# a container for get_result function below. Works just like tuple, but prettier
ActionResult = namedtuple(
    "action_result",
    ("snapshot", "observation", "reward", "terminated", "truncated", "info")
)


class WithSnapshots(Wrapper):
    """
    Creates a wrapper that supports saving and loading environemnt states.
    Required for planning algorithms.

    This class will have access to the core environment as self.env, e.g.:
    - self.env.reset()           #reset original env
    - self.env.ale.cloneState()  #make snapshot for atari. load with .restoreState()
    - ...

    You can also use reset() and step() directly for convenience.
    - s, info = self.reset()                   # same as self.env.reset()
    - s, r, terminated, truncated, info = self.step(action)  # same as self.env.step(action)

    Note that while you may use self.render(), it will spawn a window that cannot be pickled.
    Thus, you will need to call self.close() before pickling will work again.
    """

    def get_snapshot(self, render=False):
        """
        :returns: environment state that can be loaded with load_snapshot
        Snapshots guarantee same env behaviour each time they are loaded.

        Warning! Snapshots can be arbitrary things (strings, integers, json, tuples)
        Don't count on them being pickle strings when implementing MCTS.

        Developer Note: Make sure the object you return will not be affected by
        anything that happens to the environment after it's saved.
        You shouldn't, for example, return self.env.
        In case of doubt, use pickle.dumps or deepcopy.

        """
        if render:
            self.render()  # close popup windows since we can't pickle them
            self.close()

        # if self.unwrapped.viewer is not None:
        #     self.unwrapped.viewer.close()
        #     self.unwrapped.viewer = None
        return dumps(self.env)

    def load_snapshot(self, snapshot, render=False):
        """
        Loads snapshot as current env state.
        Should not change snapshot inplace (in case of doubt, deepcopy).
        """

        assert not hasattr(self, "_monitor") or hasattr(
            self.env, "_monitor"), "can't backtrack while recording"

        if render:
            self.render()  # close popup windows since we can't load into them
            self.close()
        self.env = loads(snapshot)

    def step_with_snapshot(self, snapshot, action):
        """
        A convenience function that
        - loads snapshot,
        - commits action via self.step,
        - and takes snapshot again :)

        :returns: next snapshot, next_observation, reward, is_done, info

        Basically it returns next snapshot and everything that env.step would have returned.
        """

        # load, commit, take snapshot
        ####### Здесь ваш код ########
        self.load_snapshot(snapshot)
        next_observation, reward, terminated, truncated, info = self.env.step(action)
        next_snapshot = self.get_snapshot()
        ##############################

        return ActionResult(
            next_snapshot,
            next_observation,
            reward,
            terminated,
            truncated,
            info
        )

### Try out snapshots:
Let`s check our wrapper. At first, reset environment and save it, further randomly play some actions and restore our environment from the snapshot. It should be the same as our previous initial state.

In [ ]:
def test_snapshotting():
    # make env
    env = WithSnapshots(gym.make(env_name, max_episode_steps=100))
    env.reset()

    n_actions = env.action_space.n

    # create first snapshot
    snap0 = env.get_snapshot()
    action = env.action_space.sample()

    state, _, terminated, truncated, *_ = env.step(action)
    print(state, terminated, truncated)

    # play without making snapshots (faster)
    while True:
        if terminated:
            print("You died.")
            break
        if truncated:
            print("You survived enough!")
            break
        _, _, terminated, truncated, _ = env.step(env.action_space.sample())

    env.reset()
    # reload initial state
    env.load_snapshot(snap0)

    state1, *_ = env.step(action)
    print(state1)
    assert np.all(state == state1)
    print('Load successful!')

test_snapshotting()

[ 0.00893686 -0.23770776  0.03032947  0.28127876] False False
You died.
[ 0.00893686 -0.23770776  0.03032947  0.28127876]
Load successful!


In [ ]:
def test_step_with_snapshots():
    # make env
    env = WithSnapshots(gym.make("CartPole-v1", max_episode_steps=100))
    env.reset()

    n_actions = env.action_space.n

    # create first snapshot
    snap = env.get_snapshot()

    obs, *_ = env.step(0)
    print(obs)
    obs, *_ = env.step(0)
    print(obs)

    print('repeat')
    # get outcome (snapshot, observation, reward, is_done, info)
    snap, obs, *_ = env.step_with_snapshot(snap, 0)
    print(obs)
    snap, obs, *_ = env.step_with_snapshot(snap, 0)
    print(obs)

test_step_with_snapshots()

[ 0.03107909 -0.22496122 -0.00796834  0.32572386]
[ 0.02657987 -0.4199688  -0.00145386  0.6158833 ]
repeat
[ 0.03107909 -0.22496122 -0.00796834  0.32572386]
[ 0.02657987 -0.4199688  -0.00145386  0.6158833 ]


# MCTS: Monte-Carlo tree search


We will start by implementing the `Node` class - a simple class that acts like MCTS node and supports some of the MCTS algorithm steps.

This MCTS implementation makes some assumptions about the environment, you can find those _in the notes section at the end of the notebook_.

In [ ]:
class Node:
    """A tree node for MCTS.

    Each Node corresponds to the result of performing a particular action (self.action)
    in a particular state (self.parent), and is essentially one arm in the multi-armed bandit that
    we model in that state."""

    # metadata:
    parent = None  # parent Node
    qvalue_sum = 0.  # sum of Q-values from all visits (numerator)
    times_visited = 0  # counter of visits (denominator)

    def __init__(self, parent, action, env):
        """
        Creates and empty node with no children.
        Does so by commiting an action and recording outcome.

        :param parent: parent Node
        :param action: action to commit from parent Node
        """

        self.env = env
        self.parent: Node = parent
        self.action = action
        self.children = set()  # set of child nodes

        # get action outcome and save it
        res = self.env.step_with_snapshot(parent.snapshot, action)
        self.snapshot, self.observation, self.immediate_reward, terminated, truncated, _ = res
        self.is_done = terminated or truncated

    def is_leaf(self):
        return len(self.children) == 0

    def is_root(self):
        return self.parent is None

    def get_qvalue_estimate(self):
        return self.qvalue_sum / self.times_visited if self.times_visited != 0 else 0

    def ucb_score(self, scale=10, max_value=1e30):
        """
        Computes ucb1 upper bound using current value and visit counts for node and it's parent.

        :param scale: Multiplies upper bound by that. From Hoeffding inequality,
                      assumes reward range to be [0, scale].
        :param max_value: a value that represents infinity (for unvisited nodes).

        """

        if self.times_visited == 0:
            return max_value

        # compute ucb-1 additive component (to be added to mean value)
        # hint: you can use self.parent
        # U = <YOUR CODE>
        ####### Здесь ваш код ########
        N_tot = self.parent.times_visited
        n_a = self.times_visited
        U = np.sqrt(2 * np.log(N_tot) / n_a)
        ##############################

        return self.get_qvalue_estimate() + scale * U

    # MCTS steps

    def select_best_child(self, greedy: bool):
        best_child, best_score = None, None
        for child_node in self.children:
            score = child_node.get_qvalue_estimate() if greedy else child_node.ucb_score()

            if best_child is None or score > best_score:
                best_child = child_node
                best_score = score

        return best_child


    def select_best_leaf(self):
        """
        Picks the leaf with the highest priority to expand.
        Does so by recursively picking nodes with the best UCB-1 score until it reaches a leaf.
        """
        if self.is_leaf():
            return self

        # Select the child node with the highest UCB score. You might want to implement some heuristics
        # to break ties in a smart way, although CartPole should work just fine without them.
        # best_child = <YOUR CODE>
        ####### Здесь ваш код ########
        best_child = self.select_best_child(greedy=False)
        ##############################

        return best_child.select_best_leaf()

    def expand(self):
        """
        Expands the current node by creating all possible child nodes.
        Then returns one of those children.
        """

        assert not self.is_done, "can't expand from terminal state"

        n_actions = self.env.action_space.n
        for action in range(n_actions):
            self.children.add(Node(self, action, self.env))

        # If you have implemented any heuristics in select_best_leaf(), they will be used here.
        # Otherwise, this is equivalent to picking some undefined newly created child node.
        return self.select_best_leaf()

    def rollout(self, t_max):
        """
        Play the game from this state to the end (done) or for t_max steps.

        On each step, pick action at random (hint: env.action_space.sample()).

        Compute sum of rewards from the current state until the end of the episode.
        Note 1: use env.action_space.sample() for picking a random action.

        """

        # set env into the appropriate state
        self.env.load_snapshot(self.snapshot)
        obs = self.observation
        is_done = self.is_done

        # perform rollout and compute reward `rollout_reward`
        ####### Здесь ваш код ########
        rollout_reward = 0
        for _ in range(t_max):
            if is_done:
                break
            action = self.env.action_space.sample()
            obs, reward, term, trunc, _ = self.env.step(action)
            rollout_reward += reward
            is_done = term or trunc
        ##############################

        return rollout_reward

    def propagate(self, child_qvalue):
        """
        Uses child Q-value (sum of rewards) to update parents recursively.
        """
        # 1) compute node Q-value `q_value`
        # 2) update self.qvalue_sum and times_visited
        # 3) propagate q_value upwards
        ####### Здесь ваш код ########
        q_value = self.immediate_reward + child_qvalue
        # update qvalue_sum and times_visited
        self.qvalue_sum += q_value
        self.times_visited += 1
        # propagate upwards
        if not self.is_root():
            self.parent.propagate(q_value)
        ##############################

    def safe_delete(self):
        """safe delete to prevent memory leak in some python versions"""
        del self.parent
        for child in self.children:
            child.safe_delete()
            del child

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
class Root(Node):
    def __init__(self, snapshot, observation, env):
        """
        creates special node that acts like tree root
        :snapshot: snapshot (from env.get_snapshot) to start planning from
        :observation: last environment observation
        """

        self.env = env
        self.parent = self.action = None
        self.children = set()  # set of child nodes

        # root: load snapshot and observation
        self.snapshot = snapshot
        self.observation = observation
        self.immediate_reward = 0
        self.is_done = False

    @staticmethod
    def from_node(node):
        """initializes node as root"""
        root = Root(node.snapshot, node.observation, node.env)
        # copy data
        copied_fields = ["qvalue_sum", "times_visited", "children", "is_done"]
        for field in copied_fields:
            setattr(root, field, getattr(node, field))
        return root

## Main MCTS loop

With all we implemented, MCTS boils down to a trivial piece of code.

In [ ]:
def plan_mcts(root: Root, n_iters, episode_max_len):
    """
    builds tree with monte-carlo tree search for n_iters iterations
    :param root: tree node to plan from
    :param n_iters: how many select-expand-simulate-propagete loops to make
    """
    for _ in range(n_iters):
        # node = <select best leaf>
        ####### Здесь ваш код ########
        node = root.select_best_leaf()
        ##############################

        if node.is_done:
            # All rollouts from a terminal node are empty, and thus have 0 reward.
            node.propagate(0)
            break
        else:
            # Expand the best leaf. Perform a rollout from it. Propagate the results upwards.
            # Note that here you have some leeway in choosing where to propagate from.
            # Any reasonable choice should work.
            ####### Здесь ваш код ########
            node = node.expand()
            episode_return = node.rollout(t_max=episode_max_len)
            node.propagate(episode_return)
            ##############################

## Plan and execute

Let's use our MCTS implementation to find the optimal policy.

In [ ]:
def test_mcts(env_name, init_iters, episode_max_len, n_samples_per_step):
    env = WithSnapshots(gym.make(env_name, max_episode_steps=episode_max_len))
    root_observation, _ = env.reset()
    root_snapshot = env.get_snapshot()
    root = Root(root_snapshot, root_observation, env)

    # plan from root:
    # plan_mcts(root, n_iters=init_iters, episode_max_len=episode_max_len)

    total_reward = 0  # sum of rewards
    test_env = loads(root_snapshot)  # env used to show progress

    for i in count():

        # get best child `best_child` with the highest mean reward
        ####### Здесь ваш код ########
        plan_mcts(root, n_iters=init_iters, episode_max_len=episode_max_len)
        best_child = root.select_best_child(greedy=True)
        ##############################

        # take action
        s, r, terminated, truncated, _ = test_env.step(best_child.action)

        total_reward += r
        if i % 50 == 0:
            print(f'{i:4}: {total_reward}')

        if terminated:
            print("You died.")
            break
        if truncated:
            print("You survived enough!")
            break

        print(i, root.children)
        # discard unrealized part of the tree [because not every child matters :(]
        for child in root.children:
            if child != best_child:
                child.safe_delete()

        # declare best child a new root
        root = Root.from_node(best_child)

        assert not root.is_leaf(), \
            "We ran out of tree! Need more planning! Try growing the tree right inside the loop."

        # You may want to run more planning here
        # hint: use `n_samples_per_step` param to set the number of mcts planning rounds
        # hint: use `episode_max_len` to restrict each episode max length
        ####### Здесь ваш код ########
        # plan_mcts(root, n_iters=init_iters, episode_max_len=episode_max_len)
        ##############################

    print(f'Init: {init_iters} EpL: {episode_max_len}\t SpS: {n_samples_per_step}\t R: {total_reward}')

test_mcts(
    env_name,
    init_iters=800,
    episode_max_len=500,
    n_samples_per_step=20
)

   0: 1.0
0 {<__main__.Node object at 0x7f65b73c8f10>, <__main__.Node object at 0x7f65b1fd3df0>}
1 {<__main__.Node object at 0x7f65b1fd3b80>, <__main__.Node object at 0x7f65b73c9780>}
2 {<__main__.Node object at 0x7f65b1fd3d90>, <__main__.Node object at 0x7f65b1fd3550>}
3 {<__main__.Node object at 0x7f65b1f3a800>, <__main__.Node object at 0x7f65b1f3a950>}
4 {<__main__.Node object at 0x7f65b1f3ada0>, <__main__.Node object at 0x7f65b1f3a7a0>}
5 {<__main__.Node object at 0x7f65b732a680>, <__main__.Node object at 0x7f65b1f3a8f0>}
6 {<__main__.Node object at 0x7f65b1f3aaa0>, <__main__.Node object at 0x7f65b1f3b970>}
7 {<__main__.Node object at 0x7f65b39bf940>, <__main__.Node object at 0x7f65b39bcee0>}
8 {<__main__.Node object at 0x7f65b200cd90>, <__main__.Node object at 0x7f65b200d1b0>}
9 {<__main__.Node object at 0x7f65b200cdf0>, <__main__.Node object at 0x7f65b200d0f0>}
10 {<__main__.Node object at 0x7f65b200ceb0>, <__main__.Node object at 0x7f65b200d150>}
11 {<__main__.Node object at 0x7

AssertionError: We ran out of tree! Need more planning! Try growing the tree right inside the loop.

## Bonus assignments

### Node selection and expansion

"Analyze this" assignment

UCB-1 is a weak bound as it relies on a very general bounds (Hoeffding Inequality, to be exact).
* Try playing with the exploration parameter $C_p$. The theoretically optimal $C_p$ you can get from a max reward of the environment.
* Use a different exploration strategy (bayesian UCB, for example)
* Expand not all but several random actions per `expand` call. See __the notes below__ for details.

The goal is to find out what gives the optimal performance for `CartPole-v1` for different time budgets (i.e. different n_iter in plan_mcts.)

Evaluate your results on `Acrobot-v1` - do the results change and if so, how can you explain it?

### Integrate learning into planning

Planning on each iteration is a costly thing to do. You can speed things up drastically if you train a classifier to predict which action will turn out to be best according to MCTS.

To do so, just record which action did the MCTS agent take on each step and fit something to [state, mcts_optimal_action]
* You can also use optimal actions from discarded states to get more (dirty) samples. Just don't forget to fine-tune without them.
* It's also worth a try to use P(best_action|state) from your model to select best nodes in addition to UCB
* If your model is lightweight enough, try using it as a rollout policy.

## Notes


#### Assumptions

The full list of assumptions is:

* __Finite number of actions__: we enumerate all actions in `expand`.
* __Episodic (finite) MDP__: while technically it works for infinite MDPs, we perform a rollout for $10^4$ steps. If you are knowingly infinite, please adjust `t_max` to something more reasonable.
* __Deterministic MDP__: `Node` represents the single outcome of taking `self.action` in `self.parent`, and does not support the situation where taking an action in a state may lead to different rewards and next states.
* __No discounted rewards__: we assume $\gamma=1$. If that isn't the case, you only need to change two lines in `rollout()` and use `my_qvalue = self.immediate_reward + gamma * child_qvalue` for `propagate()`.
* __pickleable env__: won't work if e.g. your env is connected to a web-browser surfing the internet. For custom envs, you may need to modify get_snapshot/load_snapshot from `WithSnapshots`.

#### On `get_best_leaf` and `expand` functions

This MCTS implementation only selects leaf nodes for expansion.
This doesn't break things down because `expand` adds all possible actions. Hence, all non-leaf nodes are by design fully expanded and shouldn't be selected.

If you want to only add a few random action on each expand, you will also have to modify `get_best_leaf` to consider returning non-leafs.

#### Rollout policy

We use a simple uniform policy for rollouts. This introduces a negative bias to good situations that can be messed up completely with random bad action. As a simple example, if you tend to rollout with uniform policy, you better don't use sharp knives and walk near cliffs.

You can improve that by integrating a reinforcement _learning_ algorithm with a computationally light agent. You can even train this agent on optimal policy found by the tree search.

#### Contributions
* Reusing some code from 5vision [solution for deephack.RL](https://github.com/5vision/uct_atari), code by Mikhail Pavlov
* Using some code from [this gist](https://gist.github.com/blole/dfebbec182e6b72ec16b66cc7e331110)

#### References
* <a id="1">[1]</a> _Coulom R. (2007) Efficient Selectivity and Backup Operators in Monte-Carlo Tree Search. In: van den Herik H.J., Ciancarini P., Donkers H.H.L.M.. (eds) Computers and Games. CG 2006. Lecture Notes in Computer Science, vol 4630. Springer, Berlin, Heidelberg_

* <a id="2">[2]</a> _Kocsis L., Szepesvári C. (2006) Bandit Based Monte-Carlo Planning. In: Fürnkranz J., Scheffer T., Spiliopoulou M. (eds) Machine Learning: ECML 2006. ECML 2006. Lecture Notes in Computer Science, vol 4212. Springer, Berlin, Heidelberg_

* <a id="3">[3]</a> _Kocsis, Levente, Csaba Szepesvári, and Jan Willemson. "Improved monte-carlo search." Univ. Tartu, Estonia, Tech. Rep 1 (2006)._

* <a id="4">[4]</a> _C. B. Browne et al., "A Survey of Monte Carlo Tree Search Methods," in IEEE Transactions on Computational Intelligence and AI in Games, vol. 4, no. 1, pp. 1-43, March 2012, doi: 10.1109/TCIAIG.2012.2186810._